In [4]:
from ortools.linear_solver import pywraplp

'''

'''

def create_data(file_path: str):

    data = {}

    with open(file_path, 'r') as f:
        if f == None:
            print('File not found')
            return None
                
        data["num_subjects"] = int(f.readline())
        data["num_students_per_subject"] = [int(x) for x in f.readline().split()]
        data["num_rooms"] = int(f.readline())
        data["num_seats_per_room"] = [int(x) for x in f.readline().split()]
        data["num_pairs"] = int(f.readline())
        data["conflicts"] = []
        for i in range (data["num_pairs"]):
            s1, s2 = [int(x) for x in f.readline().split()]
            data["conflicts"].append((s1, s2))
        
    return data

In [53]:
from ortools.linear_solver import pywraplp
import math

def solve_with_mip(num_subjects: int, num_rooms: int, nums_student_per_subject: int, 
                   num_seats_per_room: int, subject_pairs: int,
                   num_sections_per_day: int = None, num_days: int = None):
    if (not num_sections_per_day):
        num_sections_per_day = 4

    if (not num_days):
        num_days = math.ceil(num_subjects / num_sections_per_day)
    
    max_sum_sections = num_subjects
    
    # Create a new linear solver
    solver = pywraplp.Solver('Scheduling', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}

    for subject in range(num_subjects):
        for room in range(num_rooms):
            for section_id in range(max_sum_sections):
                x[(subject, room, section_id)] = solver.IntVar(0, 1, 'x[%i, %i, %i]' % (subject, room, section_id))
                
    y = solver.IntVar(0, max_sum_sections, 'y')
    
    # Constraints 1: Each subject must be assigned to a room in only one section
    # for section_id in range(max_sum_sections):
    #     solver.Add(solver.Sum([x[(subject, room, section_id)] 
    #                            for subject in range(num_subjects) 
    #                            for room in range(num_rooms)]) == 1)
    
    
    # Constraints 2: Each room must have most one subject in each section
    for room in range(num_rooms):
        for section_id in range(max_sum_sections):
            solver.Add(solver.Sum([x[(subject, room, section_id)] 
                                   for subject in range(num_subjects)]) <= 1)

    # Constraints 3: Each subject assigned to a room must have enough seats
    for subject in range(num_subjects):
        for room in range(num_rooms):
            solver.Add(solver.Sum([x[(subject, room, section_id)] * nums_student_per_subject[subject] 
                                   for section_id in range(max_sum_sections)]) <= num_seats_per_room[room])
    
    # Constraints 4: Two subjects cannot be assigned to the same room in the same section
    for room in range(num_rooms):
        for section_id in range(max_sum_sections):
            for subject1, subject2 in subject_pairs:
                solver.Add(x[(subject1, room, section_id)] + x[(subject2, room, section_id)] <= 1)
    
    # Constraints 5: Minimize the number of sections
    for subject in range(num_subjects):
        for room in range(num_rooms):
            for section_id in range(max_sum_sections):
                solver.Add(y >= x[(subject, room, section_id)] * section_id)
    
    # Objective function
    solver.Minimize(y)            
    
    # Solve the problem
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        for subject in range(num_subjects):
            for room in range(num_rooms):
                for section_id in range(max_sum_sections):
                    if x[(subject, room, section_id)].solution_value() > 0:
                        print('Subject %i is assigned to room %i in section %i' % (subject, room, section_id))
                        
    else:
        print('The problem does not have an optimal solution.')

In [54]:
data = create_data('data/data_16_3_(2).txt')

solve_with_mip(data["num_subjects"], data["num_rooms"], data["num_students_per_subject"],
               data["num_seats_per_room"], data["conflicts"])

Objective value = 0.0


In [13]:
from ortools.linear_solver import pywraplp
import math


def solve_with_ilp(num_subjects: int, num_rooms: int, subject_members: int, 
                   room_capacities: int, subject_pairs: int,
                   num_sections: int = None, num_days: int = None):
    if (not num_sections):
        num_sections = 4

    if (not num_days):
        num_days = math.ceil(num_subjects / num_sections)


    # Create a new linear solver
    solver = pywraplp.Solver('Scheduling', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # Create variables
    x = {}
    for i in range(num_subjects):
        for j in range(num_rooms):
            for k in range(num_days):
                for l in range(num_sections):
                    x[i, j, k, l] = solver.IntVar(0, 1, f'subject_{i}_room_{j}_day_{k}_section_{l}')
    
    # Set the objective function
    objective = solver.Objective()
    for j in range(num_rooms):
        for k in range(num_days):
            for l in range(num_sections):
                for i in range(num_subjects):
                    objective.SetCoefficient(x[i, j, k, l], 1)
    objective.SetMinimization()
    
    # Add the capacity constraints
    for j in range(num_rooms):
        for k in range(num_days):
            for l in range(num_sections):
                constraint = solver.Constraint(0, room_capacities[j])
                for i in range(num_subjects):
                    constraint.SetCoefficient(x[i, j, k, l], subject_members[i])

    # Add the one-subject-per-slot constraint
    for j in range(num_rooms):
        for k in range(num_days):
            for l in range(num_sections):
                constraint = solver.Constraint(1, 1)
                for i in range(num_subjects):
                    constraint.SetCoefficient(x[i, j, k, l], 1)
    
    # Add the conflict constraint
    for i, j in subject_pairs:
        for j in range(num_rooms):
            for k in range(num_days):
                for l in range(num_sections):
                    constraint = solver.Constraint(0, 1)
                    constraint.SetCoefficient(x[i, j, k, l], 1)
                    constraint.SetCoefficient(x[j, j, k, l], 1)

    # Solve the model
    status = solver.Solve()

    print('Hello')
    
    # Check if a feasible solution was found
    if status == pywraplp.Solver.OPTIMAL:
        # Print the number of days
        print(f'Number of days: {int(objective.Value())}')
        
        # Extract the schedule from the solution
        schedule = []
        for j in range(num_rooms):
            for k in range(num_days):
                for l in range(num_sections):
                    subject = None
                    for i in range(num_subjects):
                        if x[i, j, k, l].solution_value() == 1:
                            subject = i
                            break
                    schedule.append((j, k, l, subject))
        
        # Print the schedule
        print('Schedule:')
        for slot in schedule:
            room, day, section, subject = slot
            print(f'Room {room}, Day {day}, Section {section}: Subject {subject}')
    else:
        print('No feasible solution found')


In [36]:
data = create_data('data/data_10_2_(1).txt')

10 2 [33, 21, 25, 35, 27, 30, 40, 25, 33, 22] [29, 41] [(8, 4), (7, 4), (5, 4), (3, 4), (9, 4), (1, 4)] 4 3
The problem does not have an optimal solution.


In [23]:
solve_with_ilp(data["num_subjects"], data["num_rooms"], data["num_students_per_subject"],
               data["num_seats_per_room"], data["conflicts"])

Hello
Number of days: 24
Schedule:
Room 0, Day 0, Section 0: Subject 8
Room 0, Day 0, Section 1: Subject 8
Room 0, Day 0, Section 2: Subject 8
Room 0, Day 0, Section 3: Subject 8
Room 0, Day 1, Section 0: Subject 8
Room 0, Day 1, Section 1: Subject 8
Room 0, Day 1, Section 2: Subject 8
Room 0, Day 1, Section 3: Subject 8
Room 0, Day 2, Section 0: Subject 8
Room 0, Day 2, Section 1: Subject 8
Room 0, Day 2, Section 2: Subject 8
Room 0, Day 2, Section 3: Subject 8
Room 1, Day 0, Section 0: Subject 8
Room 1, Day 0, Section 1: Subject 8
Room 1, Day 0, Section 2: Subject 8
Room 1, Day 0, Section 3: Subject 8
Room 1, Day 1, Section 0: Subject 8
Room 1, Day 1, Section 1: Subject 8
Room 1, Day 1, Section 2: Subject 8
Room 1, Day 1, Section 3: Subject 8
Room 1, Day 2, Section 0: Subject 8
Room 1, Day 2, Section 1: Subject 8
Room 1, Day 2, Section 2: Subject 8
Room 1, Day 2, Section 3: Subject 8
